# Segmenting and Clustering Neighborhoods in Toronto

## Part 1: Extracting data from wikipedia

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import urllib.request

import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

In [2]:
import requests
from pandas.io.json import json_normalize

In [3]:
import folium

In [4]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

Foursquare credentials (enter your own)

In [45]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 
CLIENT_SECRET:


Scrape webpage using the BeautifulSoup package.

In [6]:
source = urllib.request.urlopen(' https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').read()
soup = BeautifulSoup(source, 'html.parser')
tables = soup.find_all('table', class_='sortable')

df = pd.DataFrame(columns=['PostalCode', 'Borough', 'Neighborhood'])
for table in tables:
    ths = table.find_all('th')
    headings = [th.text.strip() for th in ths]
    
for tr in table.find_all('tr'):
        tds = tr.find_all('td')
        if not tds:
            continue
        postal_code, borough, neighborhood = [td.text.strip() for td in tds[:3]]
        df = df.append(
            {'PostalCode': postal_code, 
             'Borough': borough, 
             'Neighborhood': neighborhood
            }, ignore_index=True
        )

Inspect data:

In [7]:
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


We start by removing all rows with unassigned borough.

In [8]:
neighborhood_data = df[df['Borough'] != 'Not assigned']
neighborhood_data.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [9]:
neighborhood_data.shape

(103, 3)

## Part 2: Enhancing with geolocation data

In [10]:
!wget -q -O 'geographical_coordinates.json' https://cocl.us/Geospatial_data
print('Data downloaded!')

Data downloaded!


Read csv file into a dataframe and rename 'Postal Code' column.

In [11]:
longitude_latitude = pd.read_csv('geographical_coordinates.json')
longitude_latitude.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)
longitude_latitude.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Join dataframes.

In [12]:
print(neighborhood_data.shape)
print(longitude_latitude.shape)

(103, 3)
(103, 3)


In [13]:
neighborhood_location = neighborhood_data.join(longitude_latitude, rsuffix='_')
neighborhood_location.drop('PostalCode_', axis=1, inplace=True)
neighborhood_location.dropna(axis=0, inplace=True)
neighborhood_location.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
2,M3A,North York,Parkwoods,43.763573,-79.188711
3,M4A,North York,Victoria Village,43.770992,-79.216917
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.773136,-79.239476
5,M6A,North York,"Lawrence Manor, Lawrence Heights",43.744734,-79.239476
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.727929,-79.262029


## Part 3: Analysis

As suggested in the assignment, the analysis will be restricted to the boroughs containing the word 'Toronto'.

In [14]:
toronto = neighborhood_location[neighborhood_location['Borough'].apply(lambda x: 'Toronto' in x)]
toronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.773136,-79.239476
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.727929,-79.262029
13,M5B,Downtown Toronto,"Garden District, Ryerson",43.781638,-79.304302
22,M5C,Downtown Toronto,St. James Town,43.770120,-79.408493
30,M4E,East Toronto,The Beaches,43.737473,-79.464763


We have geolocation data on the level of postal code, which is why the combination of postal code and borough will be chosen for the analysis as opposed to neighborhoods in the New York setting.

First we obtain the geolocation information of the city of Toronto.

In [15]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


Let us begin by visualizing the information that we have obtained so far.

In [16]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, postal_code in zip(toronto['Latitude'], toronto['Longitude'], toronto['Borough'], toronto['PostalCode']):
    label = '{}, {}'.format(postal_code, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Briefly investigate the first postal code in the list.

In [17]:
toronto.reset_index().at[0, 'PostalCode']

'M5A'

In [18]:
postal_code_latitude = toronto.reset_index().loc[0, 'Latitude'] # neighborhood latitude value
postal_code_longitude = toronto.reset_index().loc[0, 'Longitude'] # neighborhood longitude value

postal_code_name = toronto.reset_index().loc[0, 'PostalCode'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(
    postal_code_name, 
    postal_code_latitude, 
    postal_code_longitude
))

Latitude and longitude values of M5A are 43.773136, -79.23947609999999.


In [19]:
LIMIT = 100  # limit of number of venues returned by Foursquare API
radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    postal_code_latitude, 
    postal_code_longitude, 
    radius, 
    LIMIT)

Send GET request and inspect:

In [20]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f3157a6dfd3a31abbd555bc'},
 'response': {'headerLocation': 'Woburn',
  'headerFullLocation': 'Woburn, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 9,
  'suggestedBounds': {'ne': {'lat': 43.7776360045, 'lng': -79.2332557734104},
   'sw': {'lat': 43.7686359955, 'lng': -79.24569642658957}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4e261f261f6eb1ae13930699',
       'name': "Drupati's Roti & Doubles",
       'location': {'address': '1085 Bellamy Rd N',
        'crossStreet': 'Bellamy & Ellesmere',
        'lat': 43.775222138791534,
        'lng': -79.24167761001029,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.775222138791534,
          'lng': -79.24167761001029}],
        'dista

We borrow the **get_category_type** function from the Foursquare lab.

In [21]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a *pandas* dataframe.

In [22]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Drupati's Roti & Doubles,Caribbean Restaurant,43.775222,-79.241678
1,Federick Restaurant,Hakka Restaurant,43.774697,-79.241142
2,Thai One On,Thai Restaurant,43.774468,-79.241268
3,TD Canada Trust,Bank,43.774830,-79.241251
4,Centennial Recreation Centre,Athletics & Sports,43.774593,-79.236500


How many different venues has Foursquare found for this postal code?

In [23]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

9 venues were returned by Foursquare.


## Explore all postal codes
We begin by writing a function that allows us to repeat the above process for all postal codes.

In [24]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['PostalCode', 
                  'PostalCode Latitude', 
                  'PostalCode Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

We now run the code to obtain information on all postal codes.

In [25]:
toronto_venues = getNearbyVenues(names=toronto['PostalCode'],
                                   latitudes=toronto['Latitude'],
                                   longitudes=toronto['Longitude']
                                  )

M5A
M7A
M5B
M5C
M4E
M5E
M5G
M6G
M5H
M6H
M5J
M6J
M4K
M5K
M6K
M4L
M5L
M4M
M4N
M5N
M4P


Let us check how many entries this new data frame has and inspect the first few rows.

In [26]:
print(toronto_venues.shape)
toronto_venues.head()

(519, 7)


,PostalCode,PostalCode Latitude,PostalCode Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M5A,43.773136,-79.239476,Drupati's Roti & Doubles,43.775222,-79.241678,Caribbean Restaurant
1,M5A,43.773136,-79.239476,Federick Restaurant,43.774697,-79.241142,Hakka Restaurant
2,M5A,43.773136,-79.239476,Thai One On,43.774468,-79.241268,Thai Restaurant
3,M5A,43.773136,-79.239476,TD Canada Trust,43.774830,-79.241251,Bank
4,M5A,43.773136,-79.239476,Centennial Recreation Centre,43.774593,-79.236500,Athletics & Sports


Also inspect how many venues per postal Code where found.

In [27]:
toronto_venues[['PostalCode', 'Venue']].groupby('PostalCode').count()

,Venue
PostalCode,
M4E,3
M4K,34
M4L,16
M4M,36
M4P,4
M5A,9
M5B,13
M5C,33
M5E,6


And how many unique venues there are.

In [28]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 169 uniques categories.


## Analyze each postal codes

In [29]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['PostalCode'] = toronto_venues['PostalCode'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,PostalCode,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Aquarium,Art Gallery,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,M5A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,M5A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M5A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M5A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M5A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


And let's examine the new dataframe size.

In [30]:
toronto_onehot.shape

(519, 170)

Next, let's group rows by postal code and by taking the mean of the frequency of occurrence of each category

In [31]:
toronto_grouped = toronto_onehot.groupby('PostalCode').mean().reset_index()
toronto_grouped

,PostalCode,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Aquarium,Art Gallery,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,M4E,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00,0.00,...,0.00,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.00,0.000000
1,M4K,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00,0.00,...,0.00,0.00000,0.00,0.000000,0.029412,0.000000,0.000000,0.0000,0.00,0.029412
2,M4L,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00,0.00,...,0.00,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.00,0.000000
3,M4M,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00,0.00,...,0.00,0.00000,0.00,0.027778,0.000000,0.000000,0.000000,0.0000,0.00,0.027778
4,M4P,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00,0.00,...,0.00,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.00,0.000000
5,M5A,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00,0.00,...,0.00,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.00,0.000000
6,M5B,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00,0.00,...,0.00,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.00,0.000000
7,M5C,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00,0.00,...,0.00,0.00000,0.00,0.000000,0.000000,0.030303,0.000000,0.0000,0.00,0.000000
8,M5E,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00,0.00,...,0.00,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.00,0.000000
9,M5G,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00,0.00,...,0.00,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.00,0.000000


Let's confirm the new size.

In [32]:
toronto_grouped.shape

(19, 170)

Let's print each postal code along with the top 5 most common venues

In [33]:
num_top_venues = 5

for hood in toronto_grouped['PostalCode']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['PostalCode'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----M4E----
         venue  freq
0      Airport  0.33
1         Park  0.33
2     Bus Stop  0.33
3  Music Venue  0.00
4       Lounge  0.00


----M4K----
                 venue  freq
0                 Café  0.15
1                  Bar  0.06
2  Japanese Restaurant  0.06
3            Bookstore  0.06
4               Bakery  0.06


----M4L----
           venue  freq
0  Grocery Store  0.25
1           Café  0.19
2           Park  0.12
3    Coffee Shop  0.06
4          Diner  0.06


----M4M----
              venue  freq
0       Coffee Shop  0.08
1              Café  0.08
2             Diner  0.06
3               Pub  0.06
4  Sushi Restaurant  0.06


----M4P----
                 venue  freq
0  Rental Car Location  0.25
1            Drugstore  0.25
2                  Bar  0.25
3        Garden Center  0.25
4          Music Venue  0.00


----M5A----
                  venue  freq
0       Thai Restaurant  0.11
1  Caribbean Restaurant  0.11
2   Fried Chicken Joint  0.11
3                  Bank  0.11


Let's put that into a *pandas* dataframe.

First, let's write a function to sort the venues in descending order.

In [34]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [35]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['PostalCode']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
postal_code_venues_sorted = pd.DataFrame(columns=columns)
postal_code_venues_sorted['PostalCode'] = toronto_grouped['PostalCode']

for ind in np.arange(toronto_grouped.shape[0]):
    postal_code_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

postal_code_venues_sorted.head()

,PostalCode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,Airport,Park,Bus Stop,Dance Studio,Donut Shop,Doner Restaurant,Discount Store,Diner,Dessert Shop,Department Store
1,M4K,Café,Japanese Restaurant,Bar,Sandwich Place,Restaurant,Bookstore,Bakery,Italian Restaurant,Beer Bar,College Arts Building
2,M4L,Grocery Store,Café,Park,Coffee Shop,Nightclub,Italian Restaurant,Diner,Candy Store,Restaurant,Baby Store
3,M4M,Coffee Shop,Café,Diner,Sushi Restaurant,Pub,Italian Restaurant,Dessert Shop,Latin American Restaurant,Pizza Place,Smoothie Shop
4,M4P,Drugstore,Rental Car Location,Bar,Garden Center,Donut Shop,Doner Restaurant,Discount Store,Diner,Dessert Shop,Department Store


## Cluster postal codes

Run *k*-means to cluster the neighborhood into 5 clusters.

In [36]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('PostalCode', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 2, 0, 2, 4, 2, 2, 2, 0, 3], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each postal code.

In [37]:
# add clustering labels
postal_code_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(postal_code_venues_sorted.set_index('PostalCode'), on='PostalCode', how='inner')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.773136,-79.239476,2,Gas Station,Hakka Restaurant,Caribbean Restaurant,Lounge,Thai Restaurant,Fried Chicken Joint,Bank,Athletics & Sports,Bakery,Yoga Studio
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.727929,-79.262029,2,Coffee Shop,Chinese Restaurant,Convenience Store,Hobby Shop,Department Store,Deli / Bodega,Drugstore,Donut Shop,Doner Restaurant,Discount Store
13,M5B,Downtown Toronto,"Garden District, Ryerson",43.781638,-79.304302,2,Pharmacy,Pizza Place,Fast Food Restaurant,Chinese Restaurant,Gas Station,Noodle House,Bank,Fried Chicken Joint,Italian Restaurant,Thai Restaurant
22,M5C,Downtown Toronto,St. James Town,43.770120,-79.408493,2,Ramen Restaurant,Café,Coffee Shop,Sandwich Place,Sushi Restaurant,Pizza Place,Restaurant,Japanese Restaurant,Ice Cream Shop,Movie Theater
30,M4E,East Toronto,The Beaches,43.737473,-79.464763,1,Airport,Park,Bus Stop,Dance Studio,Donut Shop,Doner Restaurant,Discount Store,Diner,Dessert Shop,Department Store


Finally, let's visualize the resulting clusters

In [43]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
# add markers to the map

for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['PostalCode'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine the different postal codes

Cluster 1

In [39]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
31,Downtown Toronto,0,Grocery Store,Park,Bank,Hotel,Shopping Mall,College Gym,Drugstore,Donut Shop,Doner Restaurant,Discount Store
75,East Toronto,0,Grocery Store,Café,Park,Coffee Shop,Nightclub,Italian Restaurant,Diner,Candy Store,Restaurant,Baby Store


Cluster 2

In [40]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
30,East Toronto,1,Airport,Park,Bus Stop,Dance Studio,Donut Shop,Doner Restaurant,Discount Store,Diner,Dessert Shop,Department Store


Cluster 3

In [41]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Downtown Toronto,2,Gas Station,Hakka Restaurant,Caribbean Restaurant,Lounge,Thai Restaurant,Fried Chicken Joint,Bank,Athletics & Sports,Bakery,Yoga Studio
6,Downtown Toronto,2,Coffee Shop,Chinese Restaurant,Convenience Store,Hobby Shop,Department Store,Deli / Bodega,Drugstore,Donut Shop,Doner Restaurant,Discount Store
13,Downtown Toronto,2,Pharmacy,Pizza Place,Fast Food Restaurant,Chinese Restaurant,Gas Station,Noodle House,Bank,Fried Chicken Joint,Italian Restaurant,Thai Restaurant
22,Downtown Toronto,2,Ramen Restaurant,Café,Coffee Shop,Sandwich Place,Sushi Restaurant,Pizza Place,Restaurant,Japanese Restaurant,Ice Cream Shop,Movie Theater
41,Downtown Toronto,2,Greek Restaurant,Italian Restaurant,Coffee Shop,Furniture / Home Store,Ice Cream Shop,Restaurant,Pizza Place,Lounge,Liquor Store,Juice Bar
49,Downtown Toronto,2,Coffee Shop,Pub,Pizza Place,Restaurant,Bank,Bagel Shop,Light Rail Station,Liquor Store,Sports Bar,Sushi Restaurant
58,Downtown Toronto,2,Coffee Shop,Café,Hotel,Clothing Store,Restaurant,Gym,Steakhouse,Thai Restaurant,Bar,Bookstore
59,West Toronto,2,Coffee Shop,Aquarium,Café,Hotel,Fried Chicken Joint,Restaurant,Brewery,Scenic Lookout,Italian Restaurant,Sporting Goods Shop
66,East Toronto,2,Café,Japanese Restaurant,Bar,Sandwich Place,Restaurant,Bookstore,Bakery,Italian Restaurant,Beer Bar,College Arts Building
67,Downtown Toronto,2,Café,Coffee Shop,Vegetarian / Vegan Restaurant,Mexican Restaurant,Vietnamese Restaurant,Park,Grocery Store,Dessert Shop,Burger Joint,Bakery


Cluster 4

In [42]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
40,Downtown Toronto,3,Park,Convenience Store,Yoga Studio,Deli / Bodega,Drugstore,Donut Shop,Doner Restaurant,Discount Store,Diner,Dessert Shop
50,West Toronto,3,Park,Playground,Trail,Cosmetics Shop,Doner Restaurant,Discount Store,Diner,Dessert Shop,Department Store,Deli / Bodega


Cluster 5

In [44]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
102,Central Toronto,4,Drugstore,Rental Car Location,Bar,Garden Center,Donut Shop,Doner Restaurant,Discount Store,Diner,Dessert Shop,Department Store
